In [ ]:
import matplotlib.pyplot as plt
import numpy as np

label_font = {'fontname':'Arial', 'size':'11', 'color':'black', 'weight':'normal',
          'verticalalignment':'bottom'} 
title_font = {'fontname':'sans-serif', 'size':'16', 'color':'red', 'weight':'bold'}
legend_font = {'family':'sans-serif', 'size':'10', 'style':'normal'} 


In [ ]:
 """
    Generate samples of normally distributed white noise
    
    :param mu: Mean of the white noise
    :type mu: float
    :param sigma: Standard deviation of the noise 
    :type sigma: float
    :param size: Number of samples to generate
    :type size: integer
    :return: Array of normally distributed data
    :rtype: numpy array
"""
def genWhiteNoise(mu, sigma, size):
    samples = np.random.normal(mu, sigma, size)
    return samples

 """
    Generate time samples with jitter
    
    :param fs: Sampling frequency (in Hz)
    :type fs: float
    :param size: Number of samples to generate
    :type size: integer
    :return: Array of time samples
    :rtype: numpy array
"""
def genTimeVals(fs, size):
    uncertainties =  np.random.normal(0, 0, size = size)
    timeVals = np.asarray(range(int(size)))/(uncertainties + fs)
    return timeVals


 """
    Generate sinusoidal data

    :param amplitude: Amplitude
    :type amplitude: float
    :param frequency: Frequency of sine wave
    :type frequency: float
    :param time: Times to sample sine wave at
    :type time: numpy array
    :return: Array of sampled sine wave
    :rtype: numpy array
"""
def genSignal(amplitude, frequency, times):
    return amplitude*np.sin(2*np.pi*frequency*times)

"""
    Check if a number is a power of 10
    
    :param value: Number to check
    :type value: float
    :return: Boolean if number is a power of 10
    :rtype: Boolean
"""
def checkPowerOf10(value):
    value = int(value)
    while (value >= 10 and value % 10 == 0):
        value /= 10
    return value == 1

def convADCCount(dynamicRange, offset, bitDepth, data):
    stepSize = dynamicRange / 2**bitDepth
    # The below makes it so that an offset of 0 centers the data at 0
    lowVal = offset - dynamicRange / 2.
    highVal = offset + dynamicRange / 2.
    ADCVals = [int((x - lowVal)/stepSize) for x in data]
    ADCVals = [x if (x > 0 and x < 2**bitDepth) else (0 if x < 0 else 2**bitDepth - 1) for x in ADCVals]
    return np.asarray(ADCVals)


In [ ]:
sigFreq = 2.345E7
numSamps = 2**24

sampFreq = 1.E9
sigAmp = 0.2

muNoise = 0
sigNoise = 0.03

timeVals = genTimeVals(sampFreq, numSamps)
sigVals = genSignal(sigAmp, sigFreq, timeVals)

noiseVals = genWhiteNoise(muNoise, sigNoise, numSamps)

dataVals = noiseVals + sigVals
plt.close('all')

# Plot the data
plt.plot(timeVals*10**6, dataVals)

# The weird symbol is the unicode character for mu
plt.xlabel(u'Time (\u03BCs)', labelpad = 15, **label_font)
plt.ylabel('Amplitude (V)', **label_font)
plt.show()


In [ ]:
# dynamic range, offset, bit depth, data
dynamicRange = 1
offset = 0
bitDepth = 8
ADCVals = convADCCount(dynamicRange, offset, bitDepth, dataVals)

# Plot the data
plt.plot(timeVals*10**6, ADCVals)

# The weird symbol is the unicode character for mu
plt.xlabel(u'Time (\u03BCs)', labelpad = 15, **label_font)
plt.ylabel('Amplitude (ADC Counts)', **label_font)
plt.show()

# Plot the data
ADCValsVolts = ADCVals*1.0*dynamicRange/2**bitDepth + offset - dynamicRange/2

plt.plot(timeVals*10**6, ADCValsVolts)

# The weird symbol is the unicode character for mu
plt.xlabel(u'Time (\u03BCs)', labelpad = 15, **label_font)
plt.ylabel('Amplitude (V)', **label_font)
plt.show()


In [ ]:
%matplotlib widget
fftLen = len(ADCVals)
freqData = (np.abs(np.fft.fft(ADCValsVolts, fftLen))[0:int(fftLen/2)+1])**2
freqData = freqData/fftLen**2
freqData = np.asarray([2*x if (counter > 0 and counter < int(fftLen/2)) else x for counter, x in enumerate(freqData)])

freqStep = sampFreq/fftLen
frequencies = np.asarray(range(0, int(fftLen/2)+1))*freqStep

freqDataLog = 10*np.log10(freqData*1000/50)

maxFreq = frequencies[np.argmax(freqDataLog[1:]) + 1]/10**6
print('MAX FREQUENCY AT ' + str(round(maxFreq, 6)) + ' MHz')
print('MAX BIN AT ' + str(np.argmax(freqDataLog[1:]) + 1))

plt.plot((frequencies/10.**6)[1:], freqDataLog[1:])


plt.xlabel('Frequency (MHz)', labelpad = 15, **label_font)
plt.ylabel(r'Power (dBm)', **label_font)
plt.show()
